In [1]:
import json
import pandas as pd
import requests

In [2]:
apikey = "blank"
quote_summary_url = "https://yfapi.net/v11/finance/quoteSummary"
trending_url = "https://yfapi.net/v1/finance/trending/US"

stock = input("Please enter a stock ticker symbol: ").upper()
headers = {
    'x-api-key': apikey
}

querystring = {"modules": "price,summaryDetail,financialData"}

try:
    response = requests.request("GET", f"{quote_summary_url}/{stock}", headers=headers, params=querystring)
    stock_json = response.json()

    if 'quoteSummary' in stock_json and stock_json['quoteSummary']['result']:
        stock_info = stock_json['quoteSummary']['result'][0]
        price_info = stock_info['price']
        summary_detail = stock_info['summaryDetail']
        financial_data = stock_info['financialData']

        ticker = price_info['symbol']
        name = price_info.get('longName', 'N/A')
        current_price = price_info.get('regularMarketPrice', {}).get('raw', 'N/A')
        target_mean_price = financial_data.get('targetMeanPrice', {}).get('raw', 'N/A')
        fifty_two_week_high = summary_detail.get('fiftyTwoWeekHigh', {}).get('raw', 'N/A')
        fifty_two_week_low = summary_detail.get('fiftyTwoWeekLow', {}).get('raw', 'N/A')        
        print(f"\n{name} (Ticker: {ticker})")
        print(f"Current Price: ${current_price}")
        print(f"Target Mean Price: ${target_mean_price}")
        print(f"52 Week High: ${fifty_two_week_high}")
        print(f"52 Week Low: ${fifty_two_week_low}")

        trending = []
        try:
            response = requests.request("GET", trending_url, headers=headers)
            trending_json = response.json()
            if 'finance' in trending_json and trending_json['finance']['result']:
                trending_stocks = trending_json['finance']['result'][0]['quotes'][:5]
                trending = [stock.get('symbol', 'N/A') for stock in trending_stocks]

                print("\nTop 5 Trending Stocks:")
                for stock_symbol in trending:
                    print(f"{stock_symbol}")
            else:
                print("\nNo trending stocks available.")
        
        except Exception as e:
            print(f"Error fetching trending stocks: {e}")

        stock_data_df = pd.DataFrame({
            "Ticker": [ticker],
            "Name": [name],
            "Current Price": [current_price],
            "Target Mean Price": [target_mean_price],
            "52 Week High": [fifty_two_week_high],
            "52 Week Low": [fifty_two_week_low],
            "Trending Stock 1": [trending[0]],
            "Trending Stock 2": [trending[1]],
            "Trending Stock 3": [trending[2]],
            "Trending Stock 4": [trending[3]],
            "Trending Stock 5": [trending[4]]
        })

        stock_data_df.to_csv('stock_data.csv', index=False)
    else:
        print(f"No data found for: {stock}")

except Exception as e:
    print(f"Error fetching data for {stock}: {e}")


Please enter a stock ticker symbol:  AAPL



Apple Inc. (Ticker: AAPL)
Current Price: $227.725
Target Mean Price: $240.58
52 Week High: $237.23
52 Week Low: $164.08

Top 5 Trending Stocks:
AMSC
PWR
VRT
SNAP
BBD
